In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
from itertools import compress
from pylab import MaxNLocator
import seaborn as sns; sns.set()
from matplotlib.colors import LogNorm
from matplotlib import gridspec
import ast
import itertools
import seaborn as sns
from sklearn.preprocessing import StandardScaler

import fastcluster
from sklearn import cluster, datasets
import scipy.cluster.hierarchy as hier
from sklearn.cluster import KMeans
import time
import sys
import pickle
import itertools
import gzip

from collections import Counter

####################################################################################################################################################################################

## [1] Load SNP genotype matrix and Annotation Files

####################################################################################################################################################################################

In [3]:
#load isolate annotation file (columns of Genotype Matrix)
isolate_annotation_DF = pd.read_pickle('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_isolate_annotation.pkl')

#load SNP annotation file (rows of Genotype Matrix) with gene annotation information
SNP_annotation_DF = pd.read_pickle('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_SNP_annotation.pkl')

#load Genotypes Matrix
genotypes_array =  np.load('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_matrix.npy')

Columns of Genotype Matrix

In [4]:
isolate_annotation_DF.head()

,lineage_1,lineage_2,lineage_3,lineage_4,lineage_5,lineage_6,lineage_7,lineage_8,lineage_9,lineage_10,lineage_11,isolate_ID
0,4,2,1,2,1,1,i3,1,NaN,NaN,NaN,SAMEA3558733
1,4,2,1,2,2,1,1,NaN,NaN,NaN,NaN,SAMN03648641
2,3,1,1,i1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SAMN03647419
3,4,2,1,2,1,1,i1,NaN,NaN,NaN,NaN,SAMEA3671418
4,1,1,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SAMN07659096


In [5]:
np.shape(isolate_annotation_DF)

(31885, 12)

Rows of Genotype Matrix

In [6]:
SNP_annotation_DF.head()

,pos,ref,alt,gene_id,gene_pos,gene_category,gene_name
0,48,C,[T],Rv0001,47.0,Essential,dnaA
1,64,G,[C],Rv0001,63.0,Essential,dnaA
2,67,G,"[A, T]",Rv0001,66.0,Essential,dnaA
3,69,C,[T],Rv0001,68.0,Essential,dnaA
4,71,C,[T],Rv0001,70.0,Essential,dnaA


In [7]:
np.shape(SNP_annotation_DF)

(794219, 7)

Genotype Matrix

In [8]:
genotypes_array

array([[1, 1, 1, ..., 1, 1, 1],
       [2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       ...,
       [1, 9, 9, ..., 1, 1, 1],
       [1, 9, 1, ..., 1, 1, 1],
       [1, 9, 1, ..., 1, 1, 1]], dtype=int8)

In [9]:
np.shape(genotypes_array)

(794219, 31885)

### Store the full lineage call for each isolate in a seperate column

In [10]:
full_lineage_calls = []

for isolate_i in isolate_annotation_DF.index:
    
    isolate_i_lineage_calls = isolate_annotation_DF.loc[isolate_i , ['lineage_1','lineage_2','lineage_3','lineage_4','lineage_5','lineage_6','lineage_7','lineage_8','lineage_9','lineage_10','lineage_11']]
    isolate_i_full_lineage_call = '.'.join(list(isolate_i_lineage_calls[np.array([type(x) == str for x in isolate_i_lineage_calls.values])]))
    full_lineage_calls.append(isolate_i_full_lineage_call)
    
isolate_annotation_DF.loc[: , 'lineage_call'] = full_lineage_calls

In [11]:
isolate_annotation_DF.head()

,lineage_1,lineage_2,lineage_3,lineage_4,lineage_5,lineage_6,lineage_7,lineage_8,lineage_9,lineage_10,lineage_11,isolate_ID,lineage_call
0,4,2,1,2,1,1,i3,1,NaN,NaN,NaN,SAMEA3558733,4.2.1.2.1.1.i3.1
1,4,2,1,2,2,1,1,NaN,NaN,NaN,NaN,SAMN03648641,4.2.1.2.2.1.1
2,3,1,1,i1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SAMN03647419,3.1.1.i1
3,4,2,1,2,1,1,i1,NaN,NaN,NaN,NaN,SAMEA3671418,4.2.1.2.1.1.i1
4,1,1,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SAMN07659096,1.1.1.2


####################################################################################################################################################################################

## [2] Cluster isolates into groups using lineage assignments

####################################################################################################################################################################################

## Lineage 1

In [12]:
L1_isolate_annotation_DF = isolate_annotation_DF[isolate_annotation_DF.lineage_1 == '1']

In [13]:
np.shape(L1_isolate_annotation_DF)

(2815, 13)

In [14]:
L1_isolate_group_series = pd.Series(dict([ [isolate_ID, isolate_group] for isolate_ID, isolate_group in zip(list(L1_isolate_annotation_DF.isolate_ID) , ['1']*len(list(L1_isolate_annotation_DF.isolate_ID))) ]))

In [15]:
L1_isolate_group_series.head()

0209688                                         1
0210028                                         1
02R1267                                         1
1479144813357T181915lib5024nextseqn0036151bp    1
1479146369267T182615lib5031nextseqn0036151bp    1
dtype: object

## Lineage 2

In [16]:
L2_isolate_annotation_DF = isolate_annotation_DF[isolate_annotation_DF.lineage_1 == '2']

In [17]:
np.shape(L2_isolate_annotation_DF)

(8090, 13)

In [18]:
L2_isolate_group_series = pd.Series(dict([ [isolate_ID, isolate_group] for isolate_ID, isolate_group in zip(list(L2_isolate_annotation_DF.isolate_ID) , ['2']*len(list(L2_isolate_annotation_DF.isolate_ID))) ]))

In [19]:
L2_isolate_group_series.head()

00R0025    2
00R0086    2
01R0774    2
01R1321    2
01R1540    2
dtype: object

## Lineage 3

In [20]:
L3_isolate_annotation_DF = isolate_annotation_DF[isolate_annotation_DF.lineage_1 == '3']

In [21]:
np.shape(L3_isolate_annotation_DF)

(3398, 13)

In [22]:
L3_isolate_list = list(L3_isolate_annotation_DF.isolate_ID)

In [23]:
L3_isolate_group_series = pd.Series(dict([ [isolate_ID, isolate_group] for isolate_ID, isolate_group in zip(list(L3_isolate_annotation_DF.isolate_ID) , ['3']*len(list(L3_isolate_annotation_DF.isolate_ID))) ]))

In [24]:
L3_isolate_group_series.head()

1738    3
1762    3
1766    3
1768    3
1770    3
dtype: object

## Lineage 4

In [25]:
L4_isolate_annotation_DF = isolate_annotation_DF[isolate_annotation_DF.lineage_1 == '4']

In [26]:
np.shape(L4_isolate_annotation_DF)

(17388, 13)

In [27]:
L4_isolate_annotation_DF.head()

,lineage_1,lineage_2,lineage_3,lineage_4,lineage_5,lineage_6,lineage_7,lineage_8,lineage_9,lineage_10,lineage_11,isolate_ID,lineage_call
0,4,2,1,2,1,1,i3,1,NaN,NaN,NaN,SAMEA3558733,4.2.1.2.1.1.i3.1
1,4,2,1,2,2,1,1,NaN,NaN,NaN,NaN,SAMN03648641,4.2.1.2.2.1.1
3,4,2,1,2,1,1,i1,NaN,NaN,NaN,NaN,SAMEA3671418,4.2.1.2.1.1.i1
5,4,1,i1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,SAMEA1119683,4.1.i1.2.1
7,4,2,1,1,1,1,1,1,i1,NaN,NaN,SAMN07373373,4.2.1.1.1.1.1.1.i1


In [28]:
L4_sublineage_call_count = pd.Series(Counter(L4_isolate_annotation_DF.lineage_call))

In [29]:
L4_sublineage_call_count

4                         457
4.1                        16
4.1.i1.1                    9
4.1.i1.1.1                 19
4.1.i1.1.1.1             2836
4.1.i1.1.1.2               76
4.1.i1.1.2                117
4.1.i1.2                    8
4.1.i1.2.1               1518
4.1.i1.2.2                 53
4.1.i2.1                   93
4.1.i2.2                   29
4.2.1.1.1.1.1.1.i1       2227
4.2.1.1.1.1.1.1.i2        382
4.2.1.1.1.1.1.2             7
4.2.1.1.1.1.2             188
4.2.1.1.1.2.1.1.1           1
4.2.1.1.1.2.1.1.1.1        65
4.2.1.1.1.2.1.1.1.1.1     430
4.2.1.1.1.2.1.1.1.2        10
4.2.1.1.1.2.1.1.2          32
4.2.1.1.1.2.1.2            25
4.2.1.1.1.2.2               1
4.2.1.1.1.2.2.1            20
4.2.1.1.1.2.2.1.1.1         5
4.2.1.1.1.2.2.1.1.1.1      72
4.2.1.1.1.2.2.1.1.1.2     176
4.2.1.1.1.2.2.1.1.2        18
4.2.1.1.1.2.2.1.2          23
4.2.1.1.1.2.2.2            32
4.2.1.1.2                 420
4.2.1.2.1                   1
4.2.1.2.1.1                62
4.2.1.2.1.

#### Scrap
- 4


#### Group A (4.1, 4.2.2)
- 4.1
- 4.1.i1.1
- 4.1.i1.1.1
- 4.1.i1.1.1.1
- 4.1.i1.1.1.2
- 4.1.i1.1.2
- 4.1.i1.2
- 4.1.i1.2.1
- 4.1.i1.2.2
- 4.1.i2.1
- 4.1.i2.2


- 4.2.2.2.2
- 4.2.2.2.1
- 4.2.2.1
- 4.2.2
- 4.2.2.2


#### Group B (4.2.1.2)
- 4.2.1.2.2.1.2
- 4.2.1.2.2.1.1
- 4.2.1.2.2.2
- 4.2.1.2.2
- 4.2.1.2.2.1
- 4.2.1.2.1.2
- 4.2.1.2.1.1
- 4.2.1.2.1.1.i1
- 4.2.1.2.1.1.i2
- 4.2.1.2.1.1.i3
- 4.2.1.2.1.1.i3.1
- 4.2.1.2.1.1.i3.2
- 4.2.1.2.1.1.i4
- 4.2.1.2.1.1.i4.1
- 4.2.1.2.1.1.i4.2
- 4.2.1.2.1


#### Group C (4.2.1.1)
- 4.2.1.1.2
- 4.2.1.1.1.2.1.1.1
- 4.2.1.1.1.2.1.1.1.1
- 4.2.1.1.1.2.1.1.1.1.1
- 4.2.1.1.1.2.1.1.1.2
- 4.2.1.1.1.2.1.1.2
- 4.2.1.1.1.2.1.2
- 4.2.1.1.1.2.2
- 4.2.1.1.1.2.2.1
- 4.2.1.1.1.2.2.1.1.1
- 4.2.1.1.1.2.2.1.1.1.1
- 4.2.1.1.1.2.2.1.1.1.2
- 4.2.1.1.1.2.2.1.1.2
- 4.2.1.1.1.2.2.1.2
- 4.2.1.1.1.2.2.2
- 4.2.1.1.1.1.2
- 4.2.1.1.1.1.1.2
- 4.2.1.1.1.1.1.1.i1
- 4.2.1.1.1.1.1.1.i2

### Scrap (most specific lineage call is *4*)

In [30]:
L4_sublineage_call_count['4']

457

In [31]:
L4_scrap_isolate_annotation_DF = isolate_annotation_DF[isolate_annotation_DF.lineage_call == '4']

In [32]:
np.shape(L4_scrap_isolate_annotation_DF)

(457, 13)

In [33]:
L4_scrap_isolate_list = list(L4_scrap_isolate_annotation_DF.isolate_ID)

In [34]:
L4_scrap_isolate_group_series = pd.Series(dict([ [isolate_ID, isolate_group] for isolate_ID, isolate_group in zip(list(L4_scrap_isolate_annotation_DF.isolate_ID) , ['4_scrap']*len(list(L4_scrap_isolate_annotation_DF.isolate_ID))) ]))

In [35]:
L4_scrap_isolate_group_series.head()

01R0909          4_scrap
1795             4_scrap
ERR036227        4_scrap
ERR037533        4_scrap
IDR1500062350    4_scrap
dtype: object

### Group A (4.1 & 4.2.2)

In [36]:
group_L4A_lineages = [
'4.1',
'4.1.i1.1',
'4.1.i1.1.1',
'4.1.i1.1.1.1',
'4.1.i1.1.1.2',
'4.1.i1.1.2',
'4.1.i1.2',
'4.1.i1.2.1',
'4.1.i1.2.2',
'4.1.i2.1',
'4.1.i2.2',
'4.2.2.2.2',
'4.2.2.2.1',
'4.2.2.1',
'4.2.2',
'4.2.2.2']

In [37]:
L4_sublineage_call_count[group_L4A_lineages]

4.1               16
4.1.i1.1           9
4.1.i1.1.1        19
4.1.i1.1.1.1    2836
4.1.i1.1.1.2      76
4.1.i1.1.2       117
4.1.i1.2           8
4.1.i1.2.1      1518
4.1.i1.2.2        53
4.1.i2.1          93
4.1.i2.2          29
4.2.2.2.2        260
4.2.2.2.1        378
4.2.2.1          406
4.2.2             14
4.2.2.2            7
dtype: int64

In [38]:
int( L4_sublineage_call_count[group_L4A_lineages].sum() )

5839

In [39]:
L4A_isolate_annotation_DF = isolate_annotation_DF[[isolate_lineage_call in group_L4A_lineages for isolate_lineage_call in isolate_annotation_DF.lineage_call]]

In [40]:
np.shape(L4A_isolate_annotation_DF)

(5839, 13)

In [41]:
L4A_isolate_list = list(L4A_isolate_annotation_DF.isolate_ID)

In [42]:
L4A_isolate_group_series = pd.Series(dict([ [isolate_ID, isolate_group] for isolate_ID, isolate_group in zip(list(L4A_isolate_annotation_DF.isolate_ID) , ['4A']*len(list(L4A_isolate_annotation_DF.isolate_ID))) ]))

In [43]:
L4A_isolate_group_series.head()

00R1156    4A
00R1566    4A
01R0153    4A
01R0185    4A
01R0244    4A
dtype: object

### Group B (4.2.1.2)

In [44]:
group_L4B_lineages = [
'4.2.1.2.2.1.2',
'4.2.1.2.2.1.1',
'4.2.1.2.2.2',
'4.2.1.2.1.2',
'4.2.1.2.1.1',
'4.2.1.2.1.1.i1',
'4.2.1.2.1.1.i2',
'4.2.1.2.1.1.i3',
'4.2.1.2.1.1.i3.1',
'4.2.1.2.1.1.i3.2',
'4.2.1.2.1.1.i4',
'4.2.1.2.1.1.i4.1',
'4.2.1.2.1.1.i4.2',
'4.2.1.2.1',
'4.2.1.2.2',
'4.2.1.2.2.1']

In [45]:
L4_sublineage_call_count[group_L4B_lineages]

4.2.1.2.2.1.2        129
4.2.1.2.2.1.1        893
4.2.1.2.2.2          185
4.2.1.2.1.2           30
4.2.1.2.1.1           62
4.2.1.2.1.1.i1      1288
4.2.1.2.1.1.i2       106
4.2.1.2.1.1.i3        29
4.2.1.2.1.1.i3.1    1743
4.2.1.2.1.1.i3.2     433
4.2.1.2.1.1.i4        26
4.2.1.2.1.1.i4.1    1918
4.2.1.2.1.1.i4.2      73
4.2.1.2.1              1
4.2.1.2.2              3
4.2.1.2.2.1           39
dtype: int64

In [46]:
int( L4_sublineage_call_count[group_L4B_lineages].sum() )

6958

In [47]:
L4B_isolate_annotation_DF = isolate_annotation_DF[[isolate_lineage_call in group_L4B_lineages for isolate_lineage_call in isolate_annotation_DF.lineage_call]]

In [48]:
np.shape(L4B_isolate_annotation_DF)

(6958, 13)

In [49]:
L4B_isolate_list = list(L4B_isolate_annotation_DF.isolate_ID)

In [50]:
L4B_isolate_group_series = pd.Series(dict([ [isolate_ID, isolate_group] for isolate_ID, isolate_group in zip(list(L4B_isolate_annotation_DF.isolate_ID) , ['4B']*len(list(L4B_isolate_annotation_DF.isolate_ID))) ]))

In [51]:
L4B_isolate_group_series.head()

00R0178    4B
00R0308    4B
00R0312    4B
00R0453    4B
01R0166    4B
dtype: object

### Group C (4.2.1.1)

In [52]:
group_L4C_lineages = [
'4.2.1.1.2',
'4.2.1.1.1.2.1.1.1',
'4.2.1.1.1.2.1.1.1.1',
'4.2.1.1.1.2.1.1.1.1.1',
'4.2.1.1.1.2.1.1.1.2',
'4.2.1.1.1.2.1.1.2',
'4.2.1.1.1.2.1.2',
'4.2.1.1.1.2.2',
'4.2.1.1.1.2.2.1',
'4.2.1.1.1.2.2.1.1.1',
'4.2.1.1.1.2.2.1.1.1.1',
'4.2.1.1.1.2.2.1.1.1.2',
'4.2.1.1.1.2.2.1.1.2',
'4.2.1.1.1.2.2.1.2',
'4.2.1.1.1.2.2.2',
'4.2.1.1.1.1.2',
'4.2.1.1.1.1.1.2',
'4.2.1.1.1.1.1.1.i1',
'4.2.1.1.1.1.1.1.i2']

In [53]:
L4_sublineage_call_count[group_L4C_lineages]

4.2.1.1.2                 420
4.2.1.1.1.2.1.1.1           1
4.2.1.1.1.2.1.1.1.1        65
4.2.1.1.1.2.1.1.1.1.1     430
4.2.1.1.1.2.1.1.1.2        10
4.2.1.1.1.2.1.1.2          32
4.2.1.1.1.2.1.2            25
4.2.1.1.1.2.2               1
4.2.1.1.1.2.2.1            20
4.2.1.1.1.2.2.1.1.1         5
4.2.1.1.1.2.2.1.1.1.1      72
4.2.1.1.1.2.2.1.1.1.2     176
4.2.1.1.1.2.2.1.1.2        18
4.2.1.1.1.2.2.1.2          23
4.2.1.1.1.2.2.2            32
4.2.1.1.1.1.2             188
4.2.1.1.1.1.1.2             7
4.2.1.1.1.1.1.1.i1       2227
4.2.1.1.1.1.1.1.i2        382
dtype: int64

In [54]:
int( L4_sublineage_call_count[group_L4C_lineages].sum() )

4134

In [55]:
L4C_isolate_annotation_DF = isolate_annotation_DF[[isolate_lineage_call in group_L4C_lineages for isolate_lineage_call in isolate_annotation_DF.lineage_call]]

In [56]:
np.shape(L4C_isolate_annotation_DF)

(4134, 13)

In [57]:
L4C_isolate_list = list(L4C_isolate_annotation_DF.isolate_ID)

In [58]:
L4C_isolate_group_series = pd.Series(dict([ [isolate_ID, isolate_group] for isolate_ID, isolate_group in zip(list(L4C_isolate_annotation_DF.isolate_ID) , ['4C']*len(list(L4C_isolate_annotation_DF.isolate_ID))) ]))

In [59]:
L4C_isolate_group_series.head()

00R0223    4C
00R0435    4C
00R1399    4C
00R1405    4C
00R1547    4C
dtype: object

## Lineage 5

In [60]:
L5_isolate_annotation_DF = isolate_annotation_DF[isolate_annotation_DF.lineage_1 == '5']

In [61]:
np.shape(L5_isolate_annotation_DF)

(98, 13)

In [62]:
L5_isolate_list = list(L5_isolate_annotation_DF.isolate_ID)

In [63]:
L5_isolate_group_series = pd.Series(dict([ [isolate_ID, isolate_group] for isolate_ID, isolate_group in zip(list(L5_isolate_annotation_DF.isolate_ID) , ['5']*len(list(L5_isolate_annotation_DF.isolate_ID))) ]))

In [64]:
L5_isolate_group_series.head()

SAMEA1019128    5
SAMEA1019276    5
SAMEA1102588    5
SAMEA1117713    5
SAMEA1118906    5
dtype: object

## Lineage 6

In [65]:
L6_isolate_annotation_DF = isolate_annotation_DF[isolate_annotation_DF.lineage_1 == '6']

In [66]:
np.shape(L6_isolate_annotation_DF)

(96, 13)

In [67]:
L6_isolate_list = list(L6_isolate_annotation_DF.isolate_ID)

In [68]:
L6_isolate_group_series = pd.Series(dict([ [isolate_ID, isolate_group] for isolate_ID, isolate_group in zip(list(L6_isolate_annotation_DF.isolate_ID) , ['6']*len(list(L6_isolate_annotation_DF.isolate_ID))) ]))

In [69]:
L6_isolate_group_series.head()

IDR1500008439    6
IDR1600019997    6
SAMEA1102811     6
SAMEA1118507     6
SAMEA1119387     6
dtype: object

####################################################################################################################################################################################

## [3] Add column to Isolate Annotation that includes group assignment for each isolate

####################################################################################################################################################################################

#### Isolate Annotation file for all isolates

In [70]:
isolate_annotation_DF.head()

,lineage_1,lineage_2,lineage_3,lineage_4,lineage_5,lineage_6,lineage_7,lineage_8,lineage_9,lineage_10,lineage_11,isolate_ID,lineage_call
0,4,2,1,2,1,1,i3,1,NaN,NaN,NaN,SAMEA3558733,4.2.1.2.1.1.i3.1
1,4,2,1,2,2,1,1,NaN,NaN,NaN,NaN,SAMN03648641,4.2.1.2.2.1.1
2,3,1,1,i1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SAMN03647419,3.1.1.i1
3,4,2,1,2,1,1,i1,NaN,NaN,NaN,NaN,SAMEA3671418,4.2.1.2.1.1.i1
4,1,1,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SAMN07659096,1.1.1.2


In [71]:
np.shape(isolate_annotation_DF)

(31885, 13)

#### Isolates broken down by group

In [72]:
np.shape(L1_isolate_group_series)

(2815,)

In [73]:
np.shape(L2_isolate_group_series)

(8090,)

In [74]:
np.shape(L3_isolate_group_series)

(3398,)

In [75]:
np.shape(L4_scrap_isolate_group_series)

(457,)

In [76]:
np.shape(L4A_isolate_group_series)

(5839,)

In [77]:
np.shape(L4B_isolate_group_series)

(6958,)

In [78]:
np.shape(L4C_isolate_group_series)

(4134,)

In [79]:
np.shape(L5_isolate_group_series)

(98,)

In [80]:
np.shape(L6_isolate_group_series)

(96,)

#### Make sure each isolate has been classified into one group and append column with group assignments to isolate annotation df

In [81]:
np.shape(L1_isolate_group_series)[0] + np.shape(L2_isolate_group_series)[0] + np.shape(L3_isolate_group_series)[0] + np.shape(L4_scrap_isolate_group_series)[0] + np.shape(L4A_isolate_group_series)[0] + np.shape(L4B_isolate_group_series)[0] + np.shape(L4C_isolate_group_series)[0] + np.shape(L5_isolate_group_series)[0] + np.shape(L6_isolate_group_series)[0]

31885

In [82]:
isolate_group_series = L1_isolate_group_series.append(L2_isolate_group_series.append(L3_isolate_group_series.append(L4_scrap_isolate_group_series.append(L4A_isolate_group_series.append(L4B_isolate_group_series.append(L4C_isolate_group_series.append(L5_isolate_group_series.append(L6_isolate_group_series))))))))

In [83]:
np.shape(isolate_group_series)

(31885,)

In [84]:
isolate_group_series.head()

0209688                                         1
0210028                                         1
02R1267                                         1
1479144813357T181915lib5024nextseqn0036151bp    1
1479146369267T182615lib5031nextseqn0036151bp    1
dtype: object

In [85]:
isolate_group_series.tail()

SAMN09090627    6
SAMN09101482    6
SAMN09101505    6
SAMN09101520    6
SAMN09102367    6
dtype: object

In [86]:
isolate_annotation_DF.loc[: , 'group'] = list(isolate_group_series[isolate_annotation_DF.isolate_ID])

In [87]:
isolate_annotation_DF.head()

,lineage_1,lineage_2,lineage_3,lineage_4,lineage_5,lineage_6,lineage_7,lineage_8,lineage_9,lineage_10,lineage_11,isolate_ID,lineage_call,group
0,4,2,1,2,1,1,i3,1,NaN,NaN,NaN,SAMEA3558733,4.2.1.2.1.1.i3.1,4B
1,4,2,1,2,2,1,1,NaN,NaN,NaN,NaN,SAMN03648641,4.2.1.2.2.1.1,4B
2,3,1,1,i1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SAMN03647419,3.1.1.i1,3
3,4,2,1,2,1,1,i1,NaN,NaN,NaN,NaN,SAMEA3671418,4.2.1.2.1.1.i1,4B
4,1,1,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SAMN07659096,1.1.1.2,1


In [88]:
isolate_annotation_DF.tail()

,lineage_1,lineage_2,lineage_3,lineage_4,lineage_5,lineage_6,lineage_7,lineage_8,lineage_9,lineage_10,lineage_11,isolate_ID,lineage_call,group
31880,4,2,1,2,1,1,i3,1,NaN,NaN,NaN,SAMEA1530754,4.2.1.2.1.1.i3.1,4B
31881,4,2,1,2,1,1,i4,1,NaN,NaN,NaN,SAMEA104708028,4.2.1.2.1.1.i4.1,4B
31882,2,2,1,1,1,i3,NaN,NaN,NaN,NaN,NaN,SAMN02628527,2.2.1.1.1.i3,2
31883,4,2,1,2,1,1,i3,1,NaN,NaN,NaN,SAMEA1465982,4.2.1.2.1.1.i3.1,4B
31884,4,2,1,1,1,1,1,1,i1,NaN,NaN,SAMEA1523377,4.2.1.1.1.1.1.1.i1,4C


####################################################################################################################################################################################

## [4] *FILTER*: Drop Isolates that are typed *only* as Lineage 4 (no sub-lineage called)

####################################################################################################################################################################################

Create a boolean filter of all isolates to retain (drop isolates that were only typed as Lineage **4**, no sub-lineage)

In [89]:
isolates_to_keep_filter = [ not typed_L4_only for typed_L4_only in list(isolate_annotation_DF.group == '4_scrap') ]

In [90]:
np.sum(isolates_to_keep_filter)

31428

In [91]:
#filter Isolate annotation DF
isolate_annotation_DF = isolate_annotation_DF[isolates_to_keep_filter]

#reindex Isolate annotation DF
isolate_annotation_DF.reset_index(drop = True , inplace = True)

#filter Genotype Matrix 
genotypes_array = genotypes_array[: , np.array(isolates_to_keep_filter) ]

In [92]:
isolate_annotation_DF.head()

,lineage_1,lineage_2,lineage_3,lineage_4,lineage_5,lineage_6,lineage_7,lineage_8,lineage_9,lineage_10,lineage_11,isolate_ID,lineage_call,group
0,4,2,1,2,1,1,i3,1,NaN,NaN,NaN,SAMEA3558733,4.2.1.2.1.1.i3.1,4B
1,4,2,1,2,2,1,1,NaN,NaN,NaN,NaN,SAMN03648641,4.2.1.2.2.1.1,4B
2,3,1,1,i1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SAMN03647419,3.1.1.i1,3
3,4,2,1,2,1,1,i1,NaN,NaN,NaN,NaN,SAMEA3671418,4.2.1.2.1.1.i1,4B
4,1,1,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SAMN07659096,1.1.1.2,1


In [93]:
np.shape(isolate_annotation_DF)

(31428, 14)

In [94]:
np.shape(genotypes_array)

(794219, 31428)

#### number of isolates present in each global lineage

In [95]:
lineage_count = pd.Series(Counter(isolate_annotation_DF.lineage_1))

In [96]:
lineage_count

1     2815
2     8090
3     3398
4    16931
5       98
6       96
dtype: int64

In [97]:
int( lineage_count.sum() )

31428

####################################################################################################################################################################################

## [5] *FILTER*: Drop SNPs where *minor* allele(s) occurs in $< 1$ isolates

####################################################################################################################################################################################

Some isolates (in which the most specific lineage call was lineage '4') were dropped. So if there were SNPs in which the alternate allele was present in 1 isolate and 1 was dropped, then there is only 0 isolates with the alternate allele left in the *Genotypes Matrix*. Let's drop all SNPs in which the minor (reference or alternate) allele is present in less than 1 isolate.

1. For each row of matrix (SNP) we will count the number of **0's**, **1's**, **2's** and **3's**.
1. For each SNP we will take the max( number of 0's , number of 1's , number of 2's , number of 3's ) to get a count of the **major allele**
1. For each SNP we will sum up the total number of isolates with a designated allele (not a 9), to get the count of **non-missing** calls
1. We will then subtract the **count of major alleles** from the **number of non-missing calls** to get the number of isolates with the minor allele(s) for each SNP.
1. We will discard all SNPs for which the minor allele(s) occurs in LESS THAN 1 isolate

In [98]:
genotypes_matrix_SNP_0_count = list( np.sum( genotypes_array == 0 , axis = 1 ) ) #A
genotypes_matrix_SNP_1_count = list( np.sum( genotypes_array == 1 , axis = 1 ) ) #C
genotypes_matrix_SNP_2_count = list( np.sum( genotypes_array == 2 , axis = 1 ) ) #G
genotypes_matrix_SNP_3_count = list( np.sum( genotypes_array == 3 , axis = 1 ) ) #T

genotypes_matrix_SNP_non_missing_count = np.sum( genotypes_array != 9 , axis = 1 ) #Non-Missing Data
genotypes_matrix_major_allele_count = np.array( [ max(zero_count , one_count , two_count , three_count) for zero_count , one_count , two_count , three_count in zip(genotypes_matrix_SNP_0_count , genotypes_matrix_SNP_1_count , genotypes_matrix_SNP_2_count , genotypes_matrix_SNP_3_count) ] )

#Number of isolates with the minor allele(s) for each SNP
genotypes_matrix_minor_alleles_count = genotypes_matrix_SNP_non_missing_count - genotypes_matrix_major_allele_count

In [99]:
genotypes_matrix_minor_alleles_count

array([  5,   5,   3, ...,   3, 505,   3])

In [100]:
len(genotypes_matrix_minor_alleles_count)

794219

#### Number of SNPs that we're going to drop because the minor (alternate or reference) allele(s) is present in $< 1$ isolate.

In [101]:
np.sum( np.array( genotypes_matrix_minor_alleles_count ) < 1 )

11654

In [102]:
#boolean list that is 'True' for all SNPs where the minor allele is present in at least 1 isolate
SNPs_to_keep_filter = list( np.array( genotypes_matrix_minor_alleles_count ) >= 1 )

SNPs that had at least 1 isolate with minor allele

In [103]:
np.sum(SNPs_to_keep_filter)

782565

In [104]:
#filter SNP annotation DF
SNP_annotation_DF = SNP_annotation_DF[SNPs_to_keep_filter]

#reindex SNP annotation DF
SNP_annotation_DF.reset_index(drop = True , inplace = True)

#filter Genotype Matrix 
genotypes_array = genotypes_array[np.array(SNPs_to_keep_filter) , : ]

In [105]:
np.shape(SNP_annotation_DF)

(782565, 7)

In [106]:
np.shape(genotypes_array)

(782565, 31428)

####################################################################################################################################################################################

## [6] Save annotation files and genotypes matrix

####################################################################################################################################################################################

### Now that we've grouped isolates and have *dropped* isolates with only lineage '4' calls and have *dropped* SNPs with minor allele count $< 1$, we save the updated Genotypes Matrix and corresponding annotation files.

#### Isolate Annotation

In [107]:
isolate_annotation_DF.head()

,lineage_1,lineage_2,lineage_3,lineage_4,lineage_5,lineage_6,lineage_7,lineage_8,lineage_9,lineage_10,lineage_11,isolate_ID,lineage_call,group
0,4,2,1,2,1,1,i3,1,NaN,NaN,NaN,SAMEA3558733,4.2.1.2.1.1.i3.1,4B
1,4,2,1,2,2,1,1,NaN,NaN,NaN,NaN,SAMN03648641,4.2.1.2.2.1.1,4B
2,3,1,1,i1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SAMN03647419,3.1.1.i1,3
3,4,2,1,2,1,1,i1,NaN,NaN,NaN,NaN,SAMEA3671418,4.2.1.2.1.1.i1,4B
4,1,1,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SAMN07659096,1.1.1.2,1


In [108]:
np.shape(isolate_annotation_DF)

(31428, 14)

In [109]:
#save isolate annotation file (columns of Genotype Matrix)
isolate_annotation_DF.to_pickle('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_isolate_annotation.pkl')

#### SNP Annotation

In [110]:
SNP_annotation_DF.head()

,pos,ref,alt,gene_id,gene_pos,gene_category,gene_name
0,48,C,[T],Rv0001,47.0,Essential,dnaA
1,64,G,[C],Rv0001,63.0,Essential,dnaA
2,67,G,"[A, T]",Rv0001,66.0,Essential,dnaA
3,69,C,[T],Rv0001,68.0,Essential,dnaA
4,71,C,[T],Rv0001,70.0,Essential,dnaA


In [111]:
np.shape(SNP_annotation_DF)

(782565, 7)

In [112]:
#save SNP annotation file (rows of Genotype Matrix)
SNP_annotation_DF.to_pickle('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_SNP_annotation.pkl')

#### Genotype Matrix

In [113]:
genotypes_array

array([[1, 1, 1, ..., 1, 1, 1],
       [2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       ...,
       [1, 9, 9, ..., 1, 1, 1],
       [1, 9, 1, ..., 1, 1, 1],
       [1, 9, 1, ..., 1, 1, 1]], dtype=int8)

In [114]:
np.shape(genotypes_array)

(782565, 31428)

In [115]:
#save Genotypes Matrix
np.save('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_matrix' , genotypes_array , allow_pickle = True)

####################################################################################################################################################################################

## [7] Create DataFrame with the functional annotation for SNPs & corresponding Amino Acid Changes

####################################################################################################################################################################################

#### This block of code will determine the functional consequence of each SNP in our genotypes array.

In [116]:
#load SNP annotation file (rows of Genotype Matrix)
SNP_annotation_DF = pd.read_pickle('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_SNP_annotation.pkl')

In [117]:
SNP_annotation_DF.head()

,pos,ref,alt,gene_id,gene_pos,gene_category,gene_name
0,48,C,[T],Rv0001,47.0,Essential,dnaA
1,64,G,[C],Rv0001,63.0,Essential,dnaA
2,67,G,"[A, T]",Rv0001,66.0,Essential,dnaA
3,69,C,[T],Rv0001,68.0,Essential,dnaA
4,71,C,[T],Rv0001,70.0,Essential,dnaA


In [118]:
np.shape(SNP_annotation_DF)

(782565, 7)

#### Create a dataframe for each unique _reference_ allele - _alternate_ allele pairing (some SNPs are not biallelic and will require multiple rows)

In [119]:
#lists that will store columns for DF
SNP_index = []
pos_list = []
ref_list = []
alt_list = []
gene_category_list = []
gene_name_list = []

for SNP_i in SNP_annotation_DF.index:
    
    SNP_i_info = SNP_annotation_DF.loc[SNP_i , :]
    
    #iterate through all alternate alleles for this SNP; bi-allelic SNPs will have just 1
    for SNP_i_alt_allele in SNP_i_info.alt:
        
        #create a unique SNP index (a key for a particular mutation), Reference Position + Alternate Allele
        SNP_key = str(SNP_i_info.pos) + '_' + SNP_i_alt_allele
        
        #store information for this specific mutation
        SNP_index.append(SNP_key)
        pos_list.append(SNP_i_info.pos)
        ref_list.append(SNP_i_info.ref)
        alt_list.append(SNP_i_alt_allele)
        gene_category_list.append(SNP_i_info.gene_category)
        gene_name_list.append(SNP_i_info.gene_name)
        
#construct DF from lists
SNP_functional_annotation_DF = pd.DataFrame(index = SNP_index)
SNP_functional_annotation_DF['pos'] = pos_list
SNP_functional_annotation_DF['ref'] = ref_list
SNP_functional_annotation_DF['alt'] = alt_list
SNP_functional_annotation_DF['gene_category'] = gene_category_list
SNP_functional_annotation_DF['gene_name'] = gene_name_list 

In [120]:
SNP_functional_annotation_DF.head()

,pos,ref,alt,gene_category,gene_name
48_T,48,C,T,Essential,dnaA
64_C,64,G,C,Essential,dnaA
67_A,67,G,A,Essential,dnaA
67_T,67,G,T,Essential,dnaA
69_T,69,C,T,Essential,dnaA


In [121]:
np.shape(SNP_functional_annotation_DF)

(844429, 5)

### Cell to Annotate SNPs

In [122]:
# Important Packages
################################################################################################################################################################################################
import os
import pandas as pd
import numpy as np
import sys
import pickle

import Bio
from Bio.Alphabet import IUPAC
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import SeqIO
from StringIO import StringIO
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.Seq import MutableSeq
################################################################################################################################################################################################


# Relevant Information for H37Rv sequence SNP functional annotation
################################################################################################################################################################################################
####### Collect all DNA and Amino Acid sequences corresponding to genes on H37Rv #######
#load reference genome and reference annotation
reference_genome = '/n/data1/hms/dbmi/farhat/bin/work-horse/bin/h37rv.fasta'
for reference_genome in SeqIO.parse(reference_genome, "fasta"):
    reference_genome.seq.alphabet = IUPAC.unambiguous_dna

reference_genome_annotation = pd.read_csv('/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/H37Rv/h37rv_genome_summary.txt', '\t').set_index('name')

####### Function to translate coding DNA sequences ####### 
def translate(gene_id, sequence):

    #find which strand the gene is located on and translate
    strand = reference_genome_annotation.loc[gene_id, 'strand']
    if strand == '+':
        protein_sequence = sequence.translate(table="Bacterial", cds=False)
    elif strand == '-':
        protein_sequence = sequence.reverse_complement().translate(table="Bacterial", cds=False)

    return protein_sequence

####### Load in dictionaries for SNP annotation #######
with open('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/pickled_files/dicts_for_SNP_annotation/H37Rv_gene_seq_records.pickle', 'rb') as handle:
    ref_gene_sequences_records = pickle.load(handle)
    
with open('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/pickled_files/dicts_for_SNP_annotation/H37Rv_protein_seq_records.pickle', 'rb') as handle:
    ref_protein_sequences_records = pickle.load(handle)
    
with open('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/pickled_files/dicts_for_SNP_annotation/H37Rv_coord_gene_mapping.pickle', 'rb') as handle:
    ReferencePosition_Gene_mapping = pickle.load(handle)
    
####### get Gene Categories #######
gene_categories = pd.read_csv('/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/CSV_files/gene_categories/gene_categories.csv').set_index('name')
gene_categories_dict = dict([gene_id , gene_category] for gene_id, gene_category in zip(list(gene_categories.gene_id) , list(gene_categories.Gene_Category)))

####### get Gene Symbols #######
gene_symbol_dict = dict([gene_id , gene_symbol] for gene_id, gene_symbol in zip(list(reference_genome_annotation.symbol.index) , list( reference_genome_annotation.symbol )))
################################################################################################################################################################################################


# Function to annotate Intergenic SNPs
################################################################################################################################################################################################
def find_flanking_genes_for_intergenic_region(intergenic_ref_pos): 

    #this function finds the genes flagging an intergenic region given a reference position

    #find gene immediately in the 5' direction
    for i in range(0 , 100000):

        #move toward 5' direction
        if ReferencePosition_Gene_mapping[intergenic_ref_pos - i] != []:

            gene_to_left = ReferencePosition_Gene_mapping[intergenic_ref_pos - i][0]
            break

    #find gene immediately in the 3' direction       
    for i in range(0 , 100000):

        #move toward 3' direction
        try:
            if ReferencePosition_Gene_mapping[intergenic_ref_pos + i] != []:

                gene_to_right = ReferencePosition_Gene_mapping[intergenic_ref_pos + i][0]
                break
        
        #KeyError means we have hit the 'end' of the chromosome, the intergenic region at then end of H37Rv in 5' > 3' orientation 
        #since TB chromosome is circular the gene to the 'right' is Rv0001    
        except KeyError:
            
            gene_to_right = 'Rv0001'
            break
    
    return gene_to_left + '_' + gene_to_right
################################################################################################################################################################################################


# Function to determine whether SNPs are Synonymous or Non-Synonymous; Returns gene coordinate, codon position, AA changes, Gene Category & Symbol
################################################################################################################################################################################################
def SNP_annotate(ref_seq_position , alt_allele_i):
    
    '''
    This function takes as input a reference position on H37Rv located within a 
    gene and an alternate allele and returns whether the base change 
    would correspond to a different Amino Acid sequence that results 
    from translating the DNA sequence into an AA sequence.
    
    '''
    gene_intergenic_id_list = []
    genomic_coord_list = []
    gene_category_list = []
    gene_symbol_list = []
    Syn_NSyn_list = []
    AA_change_list = []
    
    #get the Reference Allele from the complete H37Rv reference genome, indexing starts from 0
    ref_allele_i = reference_genome.seq[int(ref_seq_position) - 1] 
    
    #find the gene that SNP occurs on; check list corresponding to H37Rv coordinate to see if there are any genes associated with RefPosition, make sure not a tRNA
    if (len(ReferencePosition_Gene_mapping[ref_seq_position]) == 1): #and ('tRNA' not in reference_genome_annotation.loc[ReferencePosition_Gene_mapping[ref_seq_position][0], 'description']):   
        
        #retrieve gene that ReferencePosition is mapped to (i.e. SNP only corresponds to 1 gene)
        gene_intergenic_id = ReferencePosition_Gene_mapping[ref_seq_position][0]

        #find genomic coordinate of SNP relative to gene (subtract 1 since reference seq starts counting at 1)
        gene_relative_coord = (ref_seq_position - 1) - min( reference_genome_annotation.loc[gene_intergenic_id , 'chromStart'] , reference_genome_annotation.loc[gene_intergenic_id , 'chromEnd'] )

        #find the genomic coordinate (relative to the gene, in the 5' to 3' direction)
        strand = reference_genome_annotation.loc[gene_intergenic_id, 'strand']
        if strand == '+':
             genomic_5_to_3_coord = (ref_seq_position) - reference_genome_annotation.loc[gene_intergenic_id , 'chromStart']

        elif strand == '-':
             genomic_5_to_3_coord = (reference_genome_annotation.loc[gene_intergenic_id , 'chromEnd']) - (ref_seq_position-1)

        #find gene category (if one exists)
        try:
            gene_category_i = gene_categories_dict[gene_intergenic_id]
        except KeyError:
            gene_category_i = 'None'

        #find gene symbol (if one exists)
        try:
            gene_symbol_i = gene_symbol_dict[gene_intergenic_id]
        except KeyError:
            gene_symbol_i = 'None'

        #alternate allele is an actual base
        if alt_allele_i in ['A','C','G','T']:

            #translate into protein sequence with the SNP in place if not InDel or intergenic region
            SNP_change = alt_allele_i

            #ALTERNATE allele (is it Syn or NSyn?)
            #get sequence from dictionary of sequences (and convert to mutable object)
            test_gene_sequence = ref_gene_sequences_records[gene_intergenic_id].seq.tomutable()

            #change reference gene sequence by the SNP in the query sequence
            test_gene_sequence[int(gene_relative_coord)] = SNP_change

            #convert back immutable object
            test_gene_sequence = test_gene_sequence.toseq()

            #translate sequence into amino acid seq
            test_protein_sequence = translate(gene_intergenic_id , test_gene_sequence)

            #store the H37Rv AA seq to compare against
            H37Rv_AA_sequence = ref_protein_sequences_records[gene_intergenic_id].seq

            #get the codon number where the SNP occurs within
            ## take the genomic coordinate (relative to the gene, in the 5' to 3' direction), divide by 3, then take the ceiling of this number (will be fraction if SNP occurs in 1st or 2nd position on codon)
            strand = reference_genome_annotation.loc[gene_intergenic_id, 'strand']
            if strand == '+':
                 genomic_5_to_3_coord = (ref_seq_position) - reference_genome_annotation.loc[gene_intergenic_id , 'chromStart']

            elif strand == '-':
                 genomic_5_to_3_coord = (reference_genome_annotation.loc[gene_intergenic_id , 'chromEnd']) - (ref_seq_position-1)

            codon_coord = int(np.ceil( float( genomic_5_to_3_coord) / 3.0 ))

            #################################################################
            #compare to AA seq of original gene
            if test_protein_sequence == H37Rv_AA_sequence:

                try:
                    SNP_type = 'S'
                    AA_change = H37Rv_AA_sequence[codon_coord-1] + str(codon_coord) + test_protein_sequence[codon_coord-1] #get the AA before & after
                
                except IndexError:
                    SNP_type = 'None'
                    AA_change = 'None'
            else:
                
                try:
                    SNP_type = 'N'
                    AA_change = H37Rv_AA_sequence[codon_coord-1] + str(codon_coord) + test_protein_sequence[codon_coord-1] #get the AA before & after
                    
                except IndexError:
                    SNP_type = 'None'
                    AA_change = 'None'
            #################################################################

        #alternate allele is a dummy (Base Call completely supports the Reference Allele)       
        else:

            SNP_type = 'None'
            AA_change = 'None'

        #store relevant info in lists    
        gene_intergenic_id_list.append(gene_intergenic_id)
        genomic_coord_list.append(genomic_5_to_3_coord)
        gene_category_list.append(gene_category_i)
        gene_symbol_list.append(gene_symbol_i)
        Syn_NSyn_list.append(SNP_type)
        AA_change_list.append(AA_change)
    
    #if no gene in H37Rv corresponds to the Reference Position for SNP, then SNP must be intergenic
    else:
        
        gene_intergenic_id = find_flanking_genes_for_intergenic_region(ref_seq_position)
        genomic_5_to_3_coord = 'None'
        gene_category_i = 'None'
        gene_symbol_i = 'None'
        SNP_type = 'I'
        AA_change = 'None'
        
        #store relevant info in lists    
        gene_intergenic_id_list.append(gene_intergenic_id)
        genomic_coord_list.append(genomic_5_to_3_coord)
        gene_category_list.append(gene_category_i)
        gene_symbol_list.append(gene_symbol_i)
        Syn_NSyn_list.append(SNP_type)
        AA_change_list.append(AA_change)
    
    #if there is only a single gene associated with this SNP, just return the individual elememts
    if len(gene_intergenic_id_list) == 1:
        return [ref_allele_i , gene_intergenic_id , genomic_5_to_3_coord , gene_category_i , gene_symbol_i , SNP_type , AA_change]
    
    #else if there are two genes associated with this SNP, return elements for each SNP annotation in a list
    elif len(gene_intergenic_id_list) > 1:
        return [ref_allele_i , gene_intergenic_id_list , genomic_coord_list , gene_category_list , gene_symbol_list , Syn_NSyn_list , AA_change_list]
################################################################################################################################################################################################

#### Iterate through each unique Reference Allele - Alternate Allele pair and get relevant annotation

In [123]:
#create lists for each new column of the SNP functional annotation DF
gene_intergenic_id_list = []
gene_pos_list = []
SNP_type_list = []
AA_change_list = []

for SNP_i in SNP_functional_annotation_DF.index:
    
    #get the reference position & alternate allele from index key
    ref_pos_i , alt_allele_i = SNP_i.split('_')
    
    #get functional annotation for SNP
    ref_allele_i , gene_intergenic_id_i , gene_pos_i , gene_category_i , gene_symbol_i , SNP_type_i , AA_change_i = SNP_annotate(int(ref_pos_i) , alt_allele_i)
    
    #store functional annotation info in lists
    gene_intergenic_id_list.append(gene_intergenic_id_i)
    gene_pos_list.append(gene_pos_i)
    SNP_type_list.append(SNP_type_i)
    AA_change_list.append(AA_change_i)
    
#append new columns to SNP functional annotation
SNP_functional_annotation_DF['gene_id'] = gene_intergenic_id_list
SNP_functional_annotation_DF['gene_pos'] = gene_pos_list
SNP_functional_annotation_DF['SNP_type'] = SNP_type_list
SNP_functional_annotation_DF['AA_change'] = AA_change_list

/home/rv76/anaconda2/lib/python2.7/site-packages/Bio/Seq.py:2715: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


In [124]:
np.shape(SNP_functional_annotation_DF)

(844429, 9)

In [125]:
SNP_functional_annotation_DF.head()

,pos,ref,alt,gene_category,gene_name,gene_id,gene_pos,SNP_type,AA_change
48_T,48,C,T,Essential,dnaA,Rv0001,48,S,V16V
64_C,64,G,C,Essential,dnaA,Rv0001,64,N,G22R
67_A,67,G,A,Essential,dnaA,Rv0001,67,N,D23N
67_T,67,G,T,Essential,dnaA,Rv0001,67,N,D23Y
69_T,69,C,T,Essential,dnaA,Rv0001,69,S,D23D


#### Pickle functional SNP annotation for downstream analysis

__Note__: 

The gene position/coordinate for the original SNP annotation file (whose rows map to the rows of the Genotypes Matrix) starts counting from 0 and starts counting from 5' > 3' regardless of which strand the gene is transcribed from, this is to accomodate the functional SNP annotation. 

The gene position/coordinate for the functional SNP annotation (above) starts counting from 1 and starts counting in the direction that the gene is transcribed which is more intuitive for reporting purposes.

In [126]:
SNP_functional_annotation_DF.to_pickle('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_SNP_functional_annotation.pkl')